In [1]:
import numpy as np
import tensorflow as tf
from simtk import unit
from simtk import openmm

In [2]:
import sys 
sys.path.insert(0,'/home/mohit/Downloads/code_notebooks/deep_boltzmann')

In [3]:
from deep_boltzmann.models import ParticleDimer
from deep_boltzmann.networks.invertible import invnet, EnergyInvNet, create_RealNVPNet
from deep_boltzmann.sampling import GaussianPriorMCMC
from deep_boltzmann.sampling.latent_sampling import BiasedModel
from deep_boltzmann.sampling.permutation import HungarianMapper
from deep_boltzmann.util import load_obj, save_obj

Using TensorFlow backend.


In [4]:
import simtk.openmm as mm
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from simtk.openmm import app
import mdtraj

In [5]:
from sys import stdout

In [6]:
pdbFile = 'bpti/bpti.pdb'
pdb = PDBFile(pdbFile)

In [7]:
print(pdb)

In [8]:
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
# Creating System
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*nanometers)

system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints= app.AllBonds, ewaldErrorTolerance=0.0005)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)


In [9]:
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

In [11]:
simulation.minimizeEnergy(maxIterations=25)
# Adding Reporters
simulation.reporters.append(PDBReporter('output_exercise1.pdb', 5))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
potentialEnergy=True, temperature=True))
# Running simulation
simulation.step(10000)

97300,-294285.5284802108,211.9474006331351
97400,-292317.1534802108,231.55514060740447
97500,-289876.0284802108,244.04388832742794
97600,-288001.2784802108,252.6803044289763
97700,-286362.5284802108,258.90425395250935
97800,-285755.6534802108,266.15337358154244
97900,-284471.9034802108,271.9294208291797
98000,-283287.5284802108,275.338963529954
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
98000,-283287.5284802108,275.338963529954


OSError: [Errno 28] No space left on device